In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
# Presidential Results URL:  https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/VOQCHQ

results_df = pd.read_csv('./Presidential_results_2000_2020.csv')

results_df2 = results_df[['year','state','state_po','county_name','candidate','party','candidatevotes','totalvotes']]

results_df2.head(10)


,year,state,state_po,county_name,candidate,party,candidatevotes,totalvotes
0,2000,ALABAMA,AL,AUTAUGA,AL GORE,DEMOCRAT,4942,17208
1,2000,ALABAMA,AL,AUTAUGA,GEORGE W. BUSH,REPUBLICAN,11993,17208
2,2000,ALABAMA,AL,AUTAUGA,RALPH NADER,GREEN,160,17208
3,2000,ALABAMA,AL,AUTAUGA,OTHER,OTHER,113,17208
4,2000,ALABAMA,AL,BALDWIN,AL GORE,DEMOCRAT,13997,56480
5,2000,ALABAMA,AL,BALDWIN,GEORGE W. BUSH,REPUBLICAN,40872,56480
6,2000,ALABAMA,AL,BALDWIN,RALPH NADER,GREEN,1033,56480
7,2000,ALABAMA,AL,BALDWIN,OTHER,OTHER,578,56480
8,2000,ALABAMA,AL,BARBOUR,AL GORE,DEMOCRAT,5188,10395
9,2000,ALABAMA,AL,BARBOUR,GEORGE W. BUSH,REPUBLICAN,5096,10395


In [3]:
#find the winner in each county
idx = results_df2.groupby(['year','state','state_po','county_name'])['candidatevotes'].idxmax()
winner_pd = results_df2.loc[idx]

winner_pd.head(5)

,year,state,state_po,county_name,candidate,party,candidatevotes,totalvotes
1,2000,ALABAMA,AL,AUTAUGA,GEORGE W. BUSH,REPUBLICAN,11993,17208
5,2000,ALABAMA,AL,BALDWIN,GEORGE W. BUSH,REPUBLICAN,40872,56480
8,2000,ALABAMA,AL,BARBOUR,AL GORE,DEMOCRAT,5188,10395
13,2000,ALABAMA,AL,BIBB,GEORGE W. BUSH,REPUBLICAN,4273,7101
17,2000,ALABAMA,AL,BLOUNT,GEORGE W. BUSH,REPUBLICAN,12667,17973


In [4]:
#limit to california and export json

winner_ca_pd = winner_pd.loc[(winner_pd.state_po == 'CA')]
winner_ca_pd['pcttotal'] = (winner_ca_pd.candidatevotes/winner_ca_pd.totalvotes*100)
winner_ca_pd.head(10)

winner_ca_pd.to_json('presidential_results_only.json', orient='records')

C:\Users\natha\AppData\Local\Temp\ipykernel_41920\4251892655.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winner_ca_pd['pcttotal'] = (winner_ca_pd.candidatevotes/winner_ca_pd.totalvotes*100)


In [5]:
#####JONAH BELOW HERE
geoData = gpd.read_file("../data/California_County_Boundaries.geojson")
geoDataFrame = pd.DataFrame(geoData)

In [6]:
geoDataFrame['CountyName'] = geoDataFrame['CountyName'].str.upper()
geoDataFrame['county_name'] = geoDataFrame['CountyName']
geoDataFrame.head()

,OBJECTID,CountyName,AdminRegion,FireMAR,LawMAR,State_FIPS_ID,County_FIPS_ID,Shape__Area,Shape__Length,geometry,county_name
0,1,ALAMEDA,Coastal,2,2,06,001,3.082165e+09,432059.180712,"MULTIPOLYGON (((-122.26853 37.90446, -122.2682...",ALAMEDA
1,2,ALPINE,Inland,4,4,06,003,3.145871e+09,274621.121060,"POLYGON ((-119.93433 38.80882, -119.93426 38.8...",ALPINE
2,3,AMADOR,Inland,4,4,06,005,2.559998e+09,357482.565247,"POLYGON ((-120.25766 38.58033, -120.25735 38.5...",AMADOR
3,4,BUTTE,Inland,3,3,06,007,7.338660e+09,526729.272631,"POLYGON ((-121.63427 40.00127, -121.63428 40.0...",BUTTE
4,5,CALAVERAS,Inland,4,4,06,009,4.352160e+09,371781.055548,"POLYGON ((-120.20981 38.50033, -120.20961 38.5...",CALAVERAS


In [7]:
#####MERGE TWO DATAFRAMES ON COUNTY
merged_df = winner_ca_pd.merge(geoDataFrame,how = "inner",on = 'county_name')
merged_df.head()


,year,state,state_po,county_name,candidate,party,candidatevotes,totalvotes,pcttotal,OBJECTID,CountyName,AdminRegion,FireMAR,LawMAR,State_FIPS_ID,County_FIPS_ID,Shape__Area,Shape__Length,geometry
0,2000,CALIFORNIA,CA,ALAMEDA,AL GORE,DEMOCRAT,342889,494336,69.363550,1,ALAMEDA,Coastal,2,2,06,001,3.082165e+09,432059.180712,"MULTIPOLYGON (((-122.26853 37.90446, -122.2682..."
1,2004,CALIFORNIA,CA,ALAMEDA,JOHN KERRY,DEMOCRAT,422585,560742,75.361753,1,ALAMEDA,Coastal,2,2,06,001,3.082165e+09,432059.180712,"MULTIPOLYGON (((-122.26853 37.90446, -122.2682..."
2,2008,CALIFORNIA,CA,ALAMEDA,BARACK OBAMA,DEMOCRAT,489106,621029,78.757353,1,ALAMEDA,Coastal,2,2,06,001,3.082165e+09,432059.180712,"MULTIPOLYGON (((-122.26853 37.90446, -122.2682..."
3,2012,CALIFORNIA,CA,ALAMEDA,BARACK OBAMA,DEMOCRAT,469684,595642,78.853405,1,ALAMEDA,Coastal,2,2,06,001,3.082165e+09,432059.180712,"MULTIPOLYGON (((-122.26853 37.90446, -122.2682..."
4,2016,CALIFORNIA,CA,ALAMEDA,HILLARY CLINTON,DEMOCRAT,514842,654266,78.690013,1,ALAMEDA,Coastal,2,2,06,001,3.082165e+09,432059.180712,"MULTIPOLYGON (((-122.26853 37.90446, -122.2682..."


In [8]:
# creates the json object needed for state/county/year
merged_df.to_json('presidential_results.json', orient='records', default_handler=str)
